In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = 'AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
font_name = fm.FontProperties(fname=font_path).get_name()
import warnings
warnings.filterwarnings(action='ignore')

# 가중치 조금 다르게

In [2]:
# 고객 및 주거래계좌 정보(CUS_INFO.CSV)
cus= pd.read_csv('cus_info_new.csv')

# 종목정보(IEM_INFO.CSV)
iem= pd.read_csv('iem_info_20210902.csv')

# 국내주식 잔고이력(STK_BNC_HIST.CSV) _ 잔고정보 (국내 주식건에 한해) <- 여기세 종목 정보 들어있음
hist = pd.read_csv('stk_bnc_hist.csv', encoding='cp949')

# 국내주식 보유기간(STK_HLD_.CSV)
train = pd.read_csv('train_0926.csv', encoding='cp949')
test= pd.read_csv('test_0926.csv', encoding='cp949')

# submission file
submission = pd.read_csv('sample_submission.csv', encoding='cp949')

In [3]:
cus.columns=["계좌 ID","성별","연령대","투자성향","자산구간","주거래상품군","Life Style","서비스 등급","총 투자기간","주거래업종구분"]
iem.columns= ["종목코드","종목한글명","종목업종", "시가총액 규모유형","시장구분"]
hist.columns= ["계좌 ID","기준일자","종목코드","잔고수량","잔고금액","주당 액면가"]

In [4]:
hist['기준일자'] = hist['기준일자'].astype('str')
hist['기준일자'] = hist['기준일자'].apply(lambda x: str(x[0:4]) + '-' + str(x[4:6]) + '-'+ str(x[6:8]))

In [5]:
train['year']= train['매수일자'].apply(lambda x: x[0:4])

In [6]:
train_2016 = train[train['year']=='2016'].copy()
train_2016['과거 보유일'] = train_2016['보유기간(일)']*0.6

train_2017 = train[train['year']=='2017'].copy()
train_2017['과거 보유일'] = train_2017['보유기간(일)']*0.65

train_2018 = train[train['year']=='2018'].copy()
train_2018['과거 보유일'] = train_2018['보유기간(일)']*0.7

train_2019 = train[train['year']=='2019'].copy()
train_2019['과거 보유일'] = train_2019['보유기간(일)']*0.75

train_2020 = train[train['year']=='2020'].copy()
train_2020['과거 보유일'] = train_2020['보유기간(일)']*0.8

train_year =pd.concat([train_2016,train_2017,train_2018,train_2019,train_2020], ignore_index=True)
train_year["과거 보유일"] = np.trunc(train_year["과거 보유일"])
train_year = train_year.drop('year', axis=1, inplace=False)

In [9]:
train_data.head(2)

,계좌 ID,종목코드,매수일자,보유기간(일),oil_3_0.8,copper_3_0.8,base_3_0.8,exchange_3_0.8,Rank,과거 보유일,...,자산구간,주거래상품군,Life Style,서비스 등급,총 투자기간,주거래업종구분,종목한글명,종목업종,시가총액 규모유형,시장구분
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A211270,2016-03-28,40,39.59,5043.41,1.5,1165.47,938.0,24.0,...,2,2,9,5,5,8,AP위성,13,3,99
1,f431b2a7651bccfc7ce8f294fdacdf0b7b31db734be701...,A011170,2016-03-28,71,39.59,5043.41,1.5,1165.47,26.0,42.0,...,5,2,9,3,6,15,롯데케미칼,12,1,1


In [8]:
# train과 test에 고객정보(cus_info)와 주식정보(iem_info)를 추가하겠습니다.

train_data = pd.merge(train_year, cus, how = "left", on = ["계좌 ID"])
train_data = pd.merge(train_data, iem, how = "left", on = ["종목코드"])

test_data = pd.merge(test, cus, how = "left", on = ["계좌 ID"])
test_data = pd.merge(test_data, iem, how = "left", on = ["종목코드"])

In [10]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [11]:
# 추가적으로 약간의 전처리를 통해 train data와 test data를 구성하겠습니다.

hist["stk_p"] = hist["잔고금액"] / hist["잔고수량"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1)
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"])
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [12]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [13]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [14]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds


KeyboardInterrupt: 

In [ ]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [ ]:
predict

In [607]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [608]:
submission["hold_d"] = np.round(predict)

In [609]:
submission.columns=["제출ID","보유기간(일)"]

In [610]:
sub = pd.merge(submission, test[['제출ID','과거 보유일']], on=['제출ID'])
sub['차이']= sub['보유기간(일)']-sub['과거 보유일']

In [611]:
def change(x):
        if x< 0 : return 0
        elif x>146 : return 146
        else: return x

In [612]:
sub["보정"] = sub["차이"].apply(change)
sub['hold_d'] = sub['보정']+sub['과거 보유일']
sub =sub.rename(columns={'제출ID':'submit_id'})
submission = sub[['submit_id','hold_d']]

In [615]:
sub

,submit_id,보유기간(일),과거 보유일,차이,보정,hold_d
0,IDX00001,64.0,153,-89.0,0.0,153.0
1,IDX00002,332.0,335,-3.0,0.0,335.0
2,IDX00003,901.0,139,762.0,146.0,285.0
3,IDX00004,901.0,236,665.0,146.0,382.0
4,IDX00005,901.0,9,892.0,146.0,155.0
...,...,...,...,...,...,...
70591,IDX70592,2.0,23,-21.0,0.0,23.0
70592,IDX70593,941.0,103,838.0,146.0,249.0
70593,IDX70594,932.0,858,74.0,74.0,932.0
70594,IDX70595,196.0,11,185.0,146.0,157.0


In [613]:
submission

,submit_id,hold_d
0,IDX00001,153.0
1,IDX00002,335.0
2,IDX00003,285.0
3,IDX00004,382.0
4,IDX00005,155.0
...,...,...
70591,IDX70592,23.0
70592,IDX70593,249.0
70593,IDX70594,932.0
70594,IDX70595,157.0


In [614]:
submission.to_csv("sub15_3_0.6.csv", header=True, index=False, encoding='cp949')

# 모델링

In [2]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_path = 'AppleGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=18)
font_name = fm.FontProperties(fname=font_path).get_name()
import warnings
warnings.filterwarnings(action='ignore')

## scaling

In [3]:
# 고객 및 주거래계좌 정보(CUS_INFO.CSV)
cus= pd.read_csv('cus_info_new.csv')

# 종목정보(IEM_INFO.CSV)
iem= pd.read_csv('iem_info_20210902.csv')

# 국내주식 잔고이력(STK_BNC_HIST.CSV) _ 잔고정보 (국내 주식건에 한해) 
hist = pd.read_csv('stk_bnc_hist.csv', encoding='cp949')

# 국내주식 보유기간(STK_HLD_.CSV)
train = pd.read_csv('train_0930.csv', encoding='cp949')
test= pd.read_csv('test_0930.csv', encoding='cp949')

# submission file
submission = pd.read_csv('sample_submission.csv', encoding='cp949')

In [4]:
cus.columns=["계좌 ID","성별","연령대","투자성향","자산구간","주거래상품군","Life Style","서비스 등급","총 투자기간","주거래업종구분"]
iem.columns= ["종목코드","종목한글명","종목업종", "시가총액 규모유형","시장구분"]
hist.columns= ["계좌 ID","기준일자","종목코드","잔고수량","잔고금액","주당 액면가"]

In [5]:
hist['기준일자'] = hist['기준일자'].astype('str')
hist['기준일자'] = hist['기준일자'].apply(lambda x: str(x[0:4]) + '-' + str(x[4:6]) + '-'+ str(x[6:8]))

In [6]:
# 앞서 언급한 것처럼 베이스라인에서는 "hist_d" column을 임의로 생성하여 모델을 학습시키도록 하겠습니다.
# 베이스라인에서는 "hold_d"값, 즉 주식 보유기간의 0.6배에 해당하는 기간을 임의로 설정하여 "hist_d"를 생성하였습니다.
# 결국 모델은 "hist_d"만큼 주식을 보유 했을때의 "hold_d"를 예측하게 될 것입니다.

train["과거 보유일"] = train["보유기간(일)"]*0.6
train["과거 보유일"] = np.trunc(train["과거 보유일"])

In [7]:
train.head(3)

,계좌 ID,종목코드,매수일자,보유기간(일),Rank,국제유가,광물가격,기준금리,환율,과거 보유일
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,2018-07-26,11,75.0,0.931182,0.532494,0.8,0.357026,6.0
1,f431b2a7651bccfc7ce8f294fdacdf0b7b31db734be701...,A023590,2018-07-26,5,201.0,0.931182,0.532494,0.8,0.357026,3.0
2,43a0eb89f16d3a46767794dda9c31cd15e1ff9daf33332...,A010820,2018-07-26,3,758.0,0.931182,0.532494,0.8,0.357026,1.0


In [8]:
# train과 test에 고객정보(cus_info)와 주식정보(iem_info)를 추가하겠습니다.

train_data = pd.merge(train, cus, how = "left", on = ["계좌 ID"])
train_data = pd.merge(train_data, iem, how = "left", on = ["종목코드"])

test_data = pd.merge(test, cus, how = "left", on = ["계좌 ID"])
test_data = pd.merge(test_data, iem, how = "left", on = ["종목코드"])

In [9]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train_data["보유기간(일)"]
train_data.drop(["보유기간(일)"], axis = 1, inplace = True)

In [10]:
# 추가적으로 약간의 전처리를 통해 train data와 test data를 구성하겠습니다.

hist["stk_p"] = hist["잔고금액"] / hist["잔고수량"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
train_data = train_data[(train_data["매수일자"] == train_data["기준일자"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["계좌 ID", "종목코드"])
test_data = test_data[(test_data["매수일자"] == test_data["기준일자"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["계좌 ID", "종목코드", "매수일자", "기준일자"], axis = 1)
test_data = test_data.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자"], axis = 1)

from sklearn.preprocessing import LabelEncoder
L_encoder = LabelEncoder()
L_encoder.fit(iem["종목한글명"])
train_data["종목한글명"] = L_encoder.transform(train_data["종목한글명"])
test_data["종목한글명"] = L_encoder.transform(test_data["종목한글명"])

In [12]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [13]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [ ]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds


In [ ]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [ ]:
predict

In [487]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [478]:
submission["hold_d"] = np.round(predict)

In [479]:
submission.columns=["제출ID","보유기간(일)"]

In [481]:
sub = pd.merge(submission, test[['제출ID','과거 보유일']], on=['제출ID'])
sub['차이']= sub['보유기간(일)']-sub['과거 보유일']

In [482]:
def change(x):
        if x< 0 : return 0
        elif x>146 : return 146
        else: return x

In [499]:
sub["보정"] = sub["차이"].apply(change)
sub['hold_d'] = sub['보정']+sub['과거 보유일']
sub =sub.rename(columns={'제출ID':'submit_id'})
submission = sub[['submit_id','hold_d']]

In [502]:
submission

,submit_id,hold_d
0,IDX00001,153.0
1,IDX00002,335.0
2,IDX00003,285.0
3,IDX00004,382.0
4,IDX00005,155.0
...,...,...
70591,IDX70592,23.0
70592,IDX70593,249.0
70593,IDX70594,926.0
70594,IDX70595,157.0


In [503]:
submission.to_csv("sub10_14_0.8.csv", header=True, index=False, encoding='cp949')

# try 1005 화요일

In [7]:
# 국내주식 보유기간(STK_HLD_.CSV)
train = pd.read_csv('train_1005최최종.csv', encoding='cp949')
test = pd.read_csv('test_1005최최종.csv', encoding='cp949')

# submission file
submission = pd.read_csv('sample_submission.csv', encoding='cp949')

In [8]:
train["과거 보유일"] = train["보유기간(일)"]*0.6
train["과거 보유일"] = np.trunc(train["과거 보유일"])

In [9]:
# train_data에서 Y값을 추출한 후 hold_d column을 지워주겠습니다.

train_label = train["보유기간(일)"]
train.drop(["보유기간(일)"], axis = 1, inplace = True)

In [10]:
train.columns

Index(['계좌 ID', '종목코드', '매수일자', '국제유가_매수', '기준금리_매수', 'Rank', '매도일자', '매수_년',
       '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형', '시장구분', '주거래업종_매수',
       'cus_cluster_3', '환율 변화율', 'past_id', '기준일자', '잔고수량', '잔고금액', '주당 액면가',
       'stk_p', '과거 보유일'],
      dtype='object')

In [11]:
test.columns

Index(['계좌 ID', '종목코드', '매수일자', '과거 보유일', '제출ID', '보유기간(일)', '국제유가_매수',
       '기준금리_매수', 'Rank', '투자성향', '서비스 등급', '총 투자기간', '종목한글명', '시가총액 규모유형',
       '시장구분', '주거래업종_매수', 'cus_cluster_3', '매수_년', '기준일자', '잔고수량', '잔고금액',
       '주당 액면가', 'stk_p'],
      dtype='object')

In [18]:
# 추가적으로 약간의 전처리를 통해 train data와 test data를 구성하겠습니다.

train_data = train.drop(["계좌 ID", "종목코드", "매수일자", "기준일자",'환율 변화율','past_id','종목한글명','매도일자'], axis = 1)
test_data = test.drop(["계좌 ID", "종목코드", "매수일자", "제출ID","보유기간(일)", "기준일자",'종목한글명'], axis = 1)

In [21]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [22]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [23]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 300 rounds


KeyboardInterrupt: 

In [ ]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)

In [ ]:
predict

In [607]:
submission = pd.read_csv("sample_submission.csv", encoding='cp949')

In [608]:
submission["hold_d"] = np.round(predict)

In [609]:
submission.columns=["제출ID","보유기간(일)"]

In [610]:
sub = pd.merge(submission, test[['제출ID','과거 보유일']], on=['제출ID'])
sub['차이']= sub['보유기간(일)']-sub['과거 보유일']

In [611]:
# 0~146인 값만 채택, 나머지는 outlier 취급 ㅎ
def change(x):
        if x< 0 : return 0
        elif x>146 : return 146
        else: return x

In [ ]:
# outlier는 완전 146으로 하고, 나머지는 0~146으로 스케일링 조절해서 0~146 사이에 오게 만들어주기!
a= sub['차이'].quantile(0.1)
b= sub['차이'].qunatile(0.9)
def min_max(x):
    if x < a : return 0
    elif x > b : return 146
    else : return (x-a)/(b-a)*146

In [612]:
sub["보정1"] = sub["차이"].apply(change)
sub["보정2"] = sub["차이"].apply(min_max)
sub['hold_d'] = sub['보정']+sub['과거 보유일']
sub =sub.rename(columns={'제출ID':'submit_id'})
submission = sub[['submit_id','hold_d']]

In [615]:
sub

,submit_id,보유기간(일),과거 보유일,차이,보정,hold_d
0,IDX00001,64.0,153,-89.0,0.0,153.0
1,IDX00002,332.0,335,-3.0,0.0,335.0
2,IDX00003,901.0,139,762.0,146.0,285.0
3,IDX00004,901.0,236,665.0,146.0,382.0
4,IDX00005,901.0,9,892.0,146.0,155.0
...,...,...,...,...,...,...
70591,IDX70592,2.0,23,-21.0,0.0,23.0
70592,IDX70593,941.0,103,838.0,146.0,249.0
70593,IDX70594,932.0,858,74.0,74.0,932.0
70594,IDX70595,196.0,11,185.0,146.0,157.0


In [613]:
submission

,submit_id,hold_d
0,IDX00001,153.0
1,IDX00002,335.0
2,IDX00003,285.0
3,IDX00004,382.0
4,IDX00005,155.0
...,...,...
70591,IDX70592,23.0
70592,IDX70593,249.0
70593,IDX70594,932.0
70594,IDX70595,157.0


In [614]:
submission.to_csv("sub15_3_0.6.csv", header=True, index=False, encoding='cp949')